In [3]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to hide/unhide code cells."></form>''')

In [7]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

import numpy as np
import datetime as dt

import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'pandas'

In [ ]:
sp = pd.read_csv('Jira.csv')

In [ ]:
#sp.head(50)

In [ ]:
sp.columns

Index(['Issue Type', 'Issue key', 'Issue id', 'Parent id', 'Summary',
       'Assignee', 'Assignee Id', 'Reporter', 'Reporter Id', 'Priority',
       'Status', 'Resolution', 'Created', 'Updated', 'Due date', 'Labels',
       'Labels.1', 'Original estimate', 'Remaining Estimate', 'Resolved',
       'Sprint', 'Sprint.1', 'Sprint.2', 'Sprint.3', 'Sprint.4', 'Sprint.5',
       'Sprint.6', 'Sprint.7', 'Sprint.8', 'Sprint.9', 'Sprint.10',
       'Time Spent'],
      dtype='object')

In [ ]:
sprints = [s for s in sp.columns if "Sprint" in s]
sp['last_sprint'] = sp[sprints[::-1]].bfill(axis=1).iloc[:, 0]
sp['first_sprint'] = sp[sprints].bfill(axis=1).iloc[:, 0]
sp['n_sprints'] = sp[sprints].count(axis=1)
sp['Planned'] = np.where((sp['Labels']=='nao_planejado') | (sp['Labels.1']=='nao_planejado'),'nao_planejado','planejado')

# prioridades
conds = [
    sp['Priority'] == 'Highest', 
    sp['Priority'] == 'High', 
    sp['Priority'] == 'Medium',
    sp['Priority'] == 'Low',
    sp['Priority'] == 'Lowest'
]

choices = [
    '1 - Highest', 
    '2 - High', 
    '3 - Medium',
    '4 - Low',
    '5 - Lowest'
]
sp['Priority'] = np.select(conds, choices)

# categorias de cards
conds = [
    (sp['Labels'] == 'Backoffice') | (sp['Labels.1'] == 'Backoffice'), 
    (sp['Labels'] == 'Run') | (sp['Labels.1'] == 'Run'), 
    (sp['Labels'] == 'Support') | (sp['Labels.1'] == 'Support'), 
    (sp['Labels'] == 'Growth') | (sp['Labels.1'] == 'Growth')
]

choices = [
    'Backoffice', 
    'Run', 
    'Support',
    'Growth'
]
sp['Category'] = np.select(conds, choices, 'NA')

sp['Created'] = pd.to_datetime(sp['Created'], dayfirst=True)
sp['Updated'] = pd.to_datetime(sp['Updated'], dayfirst=True)

# Escorregam

In [ ]:
sprint_atual = 'PS BI Sprint 13-2023'

In [ ]:
timespent = sp.loc[(sp.last_sprint.isin([sprint_atual]) & (sp.Status!='Done') & (sp['Issue Type']=='Sub-task')), ['Parent id', 'Time Spent']]
timespent = timespent.groupby('Parent id').sum().reset_index()
timespent.fillna(0, inplace=True)
timespent.columns = ['Parent id', 'Timespent_subtask']

In [ ]:
escorrega = sp.loc[(sp.last_sprint.isin([sprint_atual]) & (sp.Status!='Done') & (sp['Issue Type']=='Story')), ['Issue id', 'Summary', 'Reporter', 'Created', 'Original estimate', 'Remaining Estimate','Priority', 'Time Spent']]
escorrega = escorrega.merge(timespent, left_on='Issue id', right_on='Parent id', how='left')
escorrega['Prazo_horas'] = round(escorrega['Original estimate']/3600,1)
escorrega['Tempo_gasto'] = round(escorrega['Timespent_subtask']/3600,1)
escorrega['Time Spent'].fillna(0, inplace=True)
escorrega['Tempo_faltante'] = escorrega['Prazo_horas'] - escorrega['Tempo_gasto'].combine_first(escorrega['Time Spent']/3600)
escorrega['Tempo_faltante'] = np.where(escorrega['Tempo_faltante']<0, escorrega['Tempo_faltante']*-1,escorrega['Tempo_faltante'])
escorrega.drop(['Original estimate',  'Parent id', 'Issue id', 'Tempo_gasto', 'Timespent_subtask', 'Time Spent', 'Remaining Estimate'], axis=1, inplace=True)
display(escorrega.sort_values(by=['Priority'], ascending=[True]))
escorrega['tipo'] = 'escorregam'

,Summary,Reporter,Created,Priority,Prazo_horas,Tempo_faltante
3,Extrações de NFs em aberto (custos médicos e comissões de parceiros externos) para carga no SAP (template Diogo),Anderson Mello,2023-06-28 10:31:00,1 - Highest,8.0,8.0
5,Divergências de Receitas - vindi vs. lake vs. dataset receitas,Diogo Couto Mundim da Fonseca,2023-05-10 14:53:00,1 - Highest,4.0,4.0
7,Relatório de vendas diárias de corretores com envio por email,Bruno Pedroso Curtarelli,2023-01-17 17:57:00,1 - Highest,15.0,15.0
0,MV - Ingestão de tabelas finais,iago talamoni,2023-07-19 09:45:00,3 - Medium,NaN,NaN
1,Ingestões MV,Anderson Mello,2023-07-03 13:56:00,3 - Medium,18.0,18.0
2,Ajustes na marcação de empresa para os planos migrados da porto para nofaro (Valdir),Valdir José dos Santos Júnior,2023-06-28 10:33:00,3 - Medium,12.0,12.0
4,Receita por plano no MV,Natália Bauer Simão,2023-06-19 13:57:00,3 - Medium,18.0,16.0
6,Dash de Vendas V7,Bruno Pedroso Curtarelli,2023-05-08 15:44:00,3 - Medium,18.0,12.0


## Adicional Para Priorização:

In [ ]:
prioriza = sp.loc[(sp['Issue Type']=='Story') & (sp.last_sprint.isin(['Refinados'])) & (~sp.Summary.isin(['Estudos','Reuniões', 'Troubleshooting'])), ['Summary', 'Reporter', 'Created', 'Original estimate', 'Remaining Estimate', 'Priority']]
prioriza['Prazo_horas'] = round(prioriza['Original estimate']/3600,1)
prioriza['Tempo_faltante'] = round(prioriza['Remaining Estimate']/3600,1)
prioriza.drop(['Original estimate', 'Remaining Estimate'], axis=1, inplace=True)
display(prioriza.sort_values(by=['Priority', 'Prazo_horas'], ascending=[True, False]))
prioriza['tipo'] = 'novos'

,Summary,Reporter,Created,Priority,Prazo_horas,Tempo_faltante
540,Tratativas de PX em solicitações de cancelamento no dashboard de churn,Valdir José dos Santos Júnior,2023-01-09 12:02:00,2 - High,60.0,60.0
406,Refatorar os dashes de Renegociação e Cancelamentos,Bruno Pedroso Curtarelli,2023-03-22 14:50:00,2 - High,30.0,30.0
130,Adição de alguns campos no Dataset de Rede,Natália Bauer Simão,2023-06-01 09:40:00,2 - High,8.0,8.0
57,Ingestão da tabela comission_rule,Anderson Mello,2023-06-28 15:21:00,2 - High,6.0,6.0
56,Ingestão de coluna na user_pet_comission,Anderson Mello,2023-06-28 15:21:00,2 - High,3.0,3.0
188,Incluir projeção diária no dash de vendas,Valdir José dos Santos Júnior,2023-05-18 16:02:00,3 - Medium,30.0,30.0
190,[Teste AB] [Squad B2C] - Análise do teste AB novo login,Natalia Prado Tiago,2023-05-18 09:33:00,3 - Medium,30.0,30.0
338,Base Clientes Banco Inter,Bruno Pedroso Curtarelli,2023-04-12 10:29:00,3 - Medium,30.0,30.0
58,Ingestão de tabelas MV (receita por plano),Anderson Mello,2023-06-28 15:20:00,3 - Medium,18.0,18.0
132,Criar um ETL para juntar na camada health os históricos do MV com os dados mais recentes,Bruno Pedroso Curtarelli,2023-05-31 15:19:00,3 - Medium,18.0,18.0


In [ ]:
prioriza.append(escorrega).to_csv(str('priorização_')+str(pd.to_datetime('today'))+str('.csv'), index=False)

# Tempo por tipo de tarefa

In [ ]:
ttf = sp.loc[(sp['Issue Type']=='Story') & (sp.Status=='Done') & (sp.Sprint.isin(['Porto.pet BI Sprint 9-2023','Porto.pet BI Sprint 10-2023','Porto.pet BI Sprint 11-2023'])) & (sp.Created.dt.year >= 2023), ['Issue key','Sprint', 'Planned', 'Category', 'Created', 'Updated', 'Original estimate', 'Remaining Estimate']]

In [ ]:
ttf['Year'] = ttf['Sprint'].apply(lambda x: x.split()[3]).apply(lambda x: x.split('-')[1])
ttf['Sprint_num'] = pd.to_numeric(ttf['Sprint'].apply(lambda x: x.split()[3]).apply(lambda x: x.split('-')[0]))

In [ ]:
dist_time = ttf.groupby(['Category', 'Sprint_num']).sum()[['Original estimate']]/3600
dist_time.reset_index(inplace=True)
dist_time.sort_values(by=['Category', 'Sprint_num'])
dist_time.pivot_table(index='Category', columns='Sprint_num', values='Original estimate', fill_value=0)

Sprint_num,9,10,11
Category,,,
Backoffice,93,3,19
NA,0,0,8
Run,74,71,9
Support,0,4,2


In [ ]:
ttf.groupby(['Planned']).sum()[['Original estimate']]/3600

,Original estimate
Planned,
nao_planejado,56.0
planejado,227.0


## Tarefas por Sprint, Status, Planejamento e Tempo gasto

In [ ]:
def sprint_agg(x):
    
    names = {
        'cards': x['Issue id'].nunique()
        , 'equipe': x['Assignee'].nunique()
        , 'cards_sem_registro_tempo': x['Time Spent'].apply(lambda x: np.isnan(x)).sum()
        , 'tempo_gasto_total': round((x['Time Spent'].sum())/3600,1)
        , 'tempo_por_tarefa': round((x['Time Spent'].sum()/x['Issue id'].nunique())/3600,1)
        , 'tempo_por_pessoa': round((x['Time Spent'].sum()/x['Assignee'].nunique())/3600,1)

    }
    
    return pd.Series(names, index=['cards'
                                   , 'equipe'
                                   , 'cards_sem_registro_tempo'
                                   , 'tempo_gasto_total'
                                   , 'tempo_por_tarefa'
                                   , 'tempo_por_pessoa'
                                  ])

sp[~sp.Summary.isin(['Troubleshooting', 'Reuniões', 'Estudos'])].groupby(['last_sprint', 'Status', 'Labels']).apply(sprint_agg)

cards  equipe  \
last_sprint                 Status             Labels                         
Porto.pet BI Sprint 1-2023  Done               nao_planejado    7.0     2.0   
Porto.pet BI Sprint 10-2023 Done               Backoffice       4.0     2.0   
                                               Run              9.0     3.0   
                                               Support          1.0     1.0   
                                               nao_planejado    5.0     2.0   
Porto.pet BI Sprint 11-2023 Aguardando retorno Backoffice       3.0     3.0   
                                               Run              2.0     1.0   
                            Done               Backoffice       6.0     2.0   
                                               Run              4.0     2.0   
                                               nao_planejado    2.0     1.0   
                            In Progress        Backoffice       3.0     2.0   
                                               Run              3.0     3.0   
                                               nao_planejado    2.0     2.0   
                            To Do              Run              1.0     1.0   
                                               nao_planejado    1.0     1.0   
Porto.pet BI Sprint 2-2023  Done               nao_planejado    8.0     4.0   
Porto.pet BI Sprint 3-2023  Done               nao_planejado    8.0     2.0   
Porto.pet BI Sprint 4-2023  Done               nao_planejado    2.0     1.0   
Porto.pet BI Sprint 5-2023  Done               nao_planejado    4.0     1.0   
Porto.pet BI Sprint 6-2023  Done               nao_planejado    5.0     3.0   
Porto.pet BI Sprint 7-2023  Done               nao_planejado    2.0     1.0   
Porto.pet BI Sprint 8-2023  Done               nao_planejado    5.0     2.0   
Porto.pet BI Sprint 9-2023  Done               Backoffice       7.0     2.0   
                                               Run             12.0     3.0   
                                               nao_planejado    3.0     1.0   
Refinados                   Aguardando retorno Run              1.0     1.0   
                            To Do              Backoffice       3.0     0.0   
                                               Run             12.0     3.0   
                                               Support          1.0     0.0   

                                                              cards_sem_registro_tempo  \
last_sprint                 Status             Labels                                    
Porto.pet BI Sprint 1-2023  Done               nao_planejado                       6.0   
Porto.pet BI Sprint 10-2023 Done               Backoffice                          2.0   
                                               Run                                 3.0   
                                               Support                             0.0   
                                               nao_planejado                       2.0   
Porto.pet BI Sprint 11-2023 Aguardando retorno Backoffice                          1.0   
                                               Run                                 2.0   
                            Done               Backoffice                          4.0   
                                               Run                                 2.0   
                                               nao_planejado                       1.0   
                            In Progress        Backoffice                          2.0   
                                               Run                                 1.0   
                                               nao_planejado                       2.0   
                            To Do              Run                                 1.0   
                                               nao_planejado                       1.0   
Porto.pet BI Sprint 2-2023  Done               nao_planejado                      

## Tempo Gasto em Estudos, Reuniões e Troubleshooting

In [ ]:
def admin_agg(x):
    
    names = {
          'tempo_gasto_total': round((x['Time Spent'].sum())/3600,1)
    }
    
    return pd.Series(names, index=['tempo_gasto_total'
                                  ])
sp[sp.Summary.isin(['Troubleshooting', 'Reuniões', 'Estudos'])].groupby(['last_sprint', 'Summary']).apply(admin_agg)

tempo_gasto_total
last_sprint                 Summary                           
Porto.pet BI Sprint 10-2023 Estudos                        0.0
                            Reuniões                      19.5
                            Troubleshooting               22.5
Porto.pet BI Sprint 4-2023  Estudos                        0.0
                            Reuniões                      57.8
                            Troubleshooting               73.2
Porto.pet BI Sprint 5-2023  Estudos                        0.0
                            Reuniões                       4.0
                            Troubleshooting                2.0

# Finalizados

In [ ]:
np = sp.loc[(sp.last_sprint.isin(['Porto.pet BI Sprint 9-2023']) & (sp.Status=='Done') & (sp['Issue Type']=='Story')), ['Summary','Assignee','Reporter', 'Created', 'Original estimate', 'Priority', 'Labels']]
np.groupby('Labels').sum()[['Original estimate']]/3600

,Original estimate
Labels,
Backoffice,48.0
Run,64.0
nao_planejado,22.0
planejado,25.0


In [ ]:
finalizados = sp.loc[(sp.last_sprint.isin(['Porto.pet BI Sprint 9-2023']) & (sp.Status=='Done') & (sp['Issue Type']=='Story')), ['Summary', 'Reporter', 'Created', 'Original estimate', 'Priority']]
finalizados['Prazo_horas'] = round(finalizados['Original estimate']/3600,1)
finalizados['Prazo_dias'] = round(finalizados['Original estimate']/21600,1)
finalizados.drop('Original estimate', axis=1, inplace=True)
display(finalizados.sort_values(by=['Priority'], ascending=[True]))

,Summary,Reporter,Created,Priority,Prazo_horas,Prazo_dias
117,Ingestão das tabelas do MV - dependencia de receitas,Bruno Pedroso Curtarelli,2023-05-08,1 - Highest,6.0,1.0
197,Ingestão da tabela refuse_procedure,iago talamoni,2023-04-18,1 - Highest,6.0,1.0
152,Inclusão do campo prestador solicitante no dts_custos_medicos_contabil,Bruno Pedroso Curtarelli,2023-04-27,1 - Highest,12.0,2.0
150,Extração de pagamentos em aberto para Corretores,Bruno Pedroso Curtarelli,2023-04-28,1 - Highest,18.0,3.0
146,Apuração Bônus Elite Novo Portifólio,Diogo Couto Mundim da Fonseca,2023-05-02,1 - Highest,21.0,3.5
131,Dash de sinistro,Natália Bauer Simão,2023-05-03,1 - Highest,0.0,0.0
116,Inclusão de glosa no dts_custos_medicos_contabil,Bruno Pedroso Curtarelli,2023-05-08,1 - Highest,4.0,0.7
94,Glosa - Validando custos_medicos com ERP,iago talamoni,2023-05-12,1 - Highest,18.0,3.0
293,Migração de dados da tabela de anexos de folha de pagamento de prestadores médicos,Fabio De Goys Vieira Costa,2023-03-14,1 - Highest,6.0,1.0
86,Pagamentos em aberto custos_medicos - Auditoria,iago talamoni,2023-05-15,1 - Highest,2.0,0.3


# Indicadores por pessoa

In [ ]:
def assignee_agg(x):
    
    names = {
        'cards_totais': x['Issue id'].nunique()
        , 'cards_sem_registro_tempo': x['Time Spent'].apply(lambda x: np.isnan(x)).sum()
        , 'tempo_estimado': round(x['Original estimate'].sum()/3600,1)
        , 'tempo_gasto_total': round((x['Time Spent'].sum())/3600,1)
        , 'tempo_por_tarefa_registrada': round((x['Time Spent'].sum()/x['Time Spent'].apply(lambda x: np.isnan(x)).sum())/3600,1)
    }
    
    return pd.Series(names, index=['cards_totais'
                                   , 'cards_sem_registro_tempo'
                                   , 'tempo_estimado'
                                   , 'tempo_gasto_total'
                                   , 'tempo_por_tarefa_registrada'
                                  ])
sp.loc[sp['last_sprint']=='Porto.pet BI Sprint 11-2023'].groupby(['Assignee', 'Status']).apply(assignee_agg)

cards_totais  \
Assignee                      Status                             
Bruno Pedroso Curtarelli      Done                         3.0   
                              To Do                        1.0   
Diogo Couto Mundim da Fonseca Aguardando retorno           7.0   
                              Done                        17.0   
                              In Progress                  5.0   
                              To Do                        4.0   
Joana Tardelli Santiago Leal  Done                         7.0   
                              In Progress                  3.0   
                              To Do                        1.0   
helena.manzan                 Aguardando retorno           3.0   
                              Done                         6.0   
                              In Progress                  4.0   
                              To Do                        4.0   
iago talamoni                 Aguardando retorno           1.0   
                              Done                        13.0   
                              In Progress                  6.0   
                              To Do                        7.0   

                                                  cards_sem_registro_tempo  \
Assignee                      Status                                         
Bruno Pedroso Curtarelli      Done                                     2.0   
                              To Do                                    1.0   
Diogo Couto Mundim da Fonseca Aguardando retorno                       6.0   
                              Done                                     2.0   
                              In Progress                              3.0   
                              To Do                                    4.0   
Joana Tardelli Santiago Leal  Done                                     3.0   
                              In Progress                              2.0   
                              To Do                                    1.0   
helena.manzan                 Aguardando retorno                       0.0   
                              Done                                     1.0   
                              In Progress                              1.0   
                              To Do                                    4.0   
iago talamoni                 Aguardando retorno                       0.0   
                              Done                                    11.0   
                              In Progress                              5.0   
                              To Do                                    7.0   

                                                  tempo_estimado  \
Assignee                      Status                               
Bruno Pedroso Curtarelli      Done                           6.0   
                              To Do                         12.0   
Diogo Couto Mundim da Fonseca Aguardando retorno            41.0   
                              Done                          41.5   
                              In Progress                   76.0   
                              To Do                          2.0   
Joana Tardelli Santiago Leal  Done                           7.0   
                              In Progress                   48.0   
                              To Do                          0.0   
helena.manzan                 Aguardando retorno            21.0   
                              Done                          40.0   
                              In Progress                   73.0   
                              To Do                          9.0   
iago talamoni                 Aguardando retorno            30.0   
                              Done                          13.0   
                              In Progress                  105.0   
                              To Do                          0.0   

                   

In [ ]:
idvars = [i for i in sp.columns if i not in sprints]
spm = sp.melt(id_vars=idvars, value_vars=sprints, value_name='sprint_name', var_name='sprint_id').dropna(subset=['sprint_name'])

# Horas estimadas por pessoa

In [ ]:
remove_sprints = ['Porto.pet BI Sprint 20', 'Porto.pet BI Sprint 21', 'Porto.pet BI Sprint 22', 'Porto.pet BI Sprint 23']
spm.loc[(spm['Issue Type']=='Story') & (~spm['sprint_name'].isin(remove_sprints))].pivot_table(index='Assignee', columns='sprint_name', values='Original estimate', aggfunc='sum')/3600

sprint_name,Porto.pet BI Sprint 1-2023,Porto.pet BI Sprint 2-2023,Porto.pet BI Sprint 3-2023,Porto.pet BI Sprint 4-2023,Porto.pet BI Sprint 5-2023,Porto.pet BI Sprint 6-2023
Assignee,,,,,,
Bruno Pedroso Curtarelli,1.0,NaN,15.0,72.0,12.0,NaN
Diogo Couto Mundim da Fonseca,14.0,0.0,0.0,12.0,0.0,24.0
Joana Tardelli Santiago Leal,54.0,108.0,78.0,102.0,72.0,12.0
Jose Antonio Lopes Junior,66.0,6.0,NaN,NaN,NaN,NaN
helena.manzan,60.0,180.0,72.0,120.0,132.0,NaN
iago talamoni,45.0,36.0,98.0,91.0,72.0,NaN


# Horas concluidas por pessoa

In [ ]:
spm.loc[(spm['Issue Type']=='Story') & (~spm['sprint_name'].isin(remove_sprints)) & (spm['Status']=='Done')].pivot_table(index='Assignee', columns='sprint_name', values='Original estimate', aggfunc='sum')/3600

sprint_name,Porto.pet BI Sprint 1-2023,Porto.pet BI Sprint 2-2023,Porto.pet BI Sprint 3-2023,Porto.pet BI Sprint 4-2023,Porto.pet BI Sprint 5-2023
Assignee,,,,,
Bruno Pedroso Curtarelli,1.0,NaN,15.0,72.0,12.0
Diogo Couto Mundim da Fonseca,14.0,0.0,0.0,0.0,NaN
Joana Tardelli Santiago Leal,54.0,108.0,54.0,30.0,NaN
Jose Antonio Lopes Junior,66.0,6.0,NaN,NaN,NaN
helena.manzan,NaN,120.0,72.0,60.0,60.0
iago talamoni,45.0,21.0,83.0,76.0,21.0


In [ ]:
spm.loc[(spm['Issue Type']=='Story') & (~spm['sprint_name'].isin(remove_sprints)) & (spm['Status']=='Done') & (spm['Assignee']=='Jose Antonio Lopes Junior')]

,Issue Type,Issue key,Issue id,Parent id,Summary,Assignee,Assignee Id,Reporter,Reporter Id,Priority,...,Labels,Original estimate,Remaining Estimate,Resolved,Time Spent,last_sprint,first_sprint,n_sprints,sprint_id,sprint_name
87,Story,PPDT-1417,178718,NaN,Atualização da base de dados de clientes Porto.Pet,Jose Antonio Lopes Junior,60fff2c1eb7aef0069849e16,Bruno Pedroso Curtarelli,5c335a512020af5dbe85b958,4 - Low,...,nao_planejado,NaN,NaN,02/Feb/23 12:11 PM,NaN,Porto.pet BI Sprint 2-2023,Porto.pet BI Sprint 2-2023,1,Sprint,Porto.pet BI Sprint 2-2023
127,Story,PPDT-1373,175106,NaN,Relatório de reajuste por idade (189) do MV,Jose Antonio Lopes Junior,60fff2c1eb7aef0069849e16,Bruno Pedroso Curtarelli,5c335a512020af5dbe85b958,4 - Low,...,planejado,21600.0,21600.0,02/Feb/23 11:32 AM,NaN,Porto.pet BI Sprint 2-2023,Porto.pet BI Sprint 2-2023,1,Sprint,Porto.pet BI Sprint 2-2023
131,Story,PPDT-1368,173838,NaN,relatório com todos os procedimentos de consulta autorizados de 01/10/2022 a 11/01/23,Jose Antonio Lopes Junior,60fff2c1eb7aef0069849e16,Bruno Pedroso Curtarelli,5c335a512020af5dbe85b958,1 - Highest,...,planejado,NaN,NaN,23/Jan/23 1:01 PM,NaN,Porto.pet BI Sprint 1-2023,Porto.pet BI Sprint 1-2023,1,Sprint,Porto.pet BI Sprint 1-2023
133,Story,PPDT-1366,173389,NaN,Ajuste no campo dt_nascimento_tutor na beneficiarios,Jose Antonio Lopes Junior,60fff2c1eb7aef0069849e16,Bruno Pedroso Curtarelli,5c335a512020af5dbe85b958,4 - Low,...,planejado,NaN,NaN,23/Jan/23 12:58 PM,NaN,Porto.pet BI Sprint 1-2023,Porto.pet BI Sprint 1-2023,1,Sprint,Porto.pet BI Sprint 1-2023
141,Story,PPDT-1358,172538,NaN,Ajustar a atribuição do plano na tabela de beneficiários,Jose Antonio Lopes Junior,60fff2c1eb7aef0069849e16,Bruno Pedroso Curtarelli,5c335a512020af5dbe85b958,1 - Highest,...,planejado,108000.0,108000.0,23/Jan/23 1:01 PM,NaN,Porto.pet BI Sprint 1-2023,Porto.pet BI Sprint 1-2023,1,Sprint,Porto.pet BI Sprint 1-2023
145,Story,PPDT-1354,172529,NaN,Inclusão da coluna prestador solicitante no dataset de custos médicos,Jose Antonio Lopes Junior,60fff2c1eb7aef0069849e16,Bruno Pedroso Curtarelli,5c335a512020af5dbe85b958,3 - Medium,...,planejado,43200.0,43200.0,23/Jan/23 1:00 PM,NaN,Porto.pet BI Sprint 1-2023,Porto.pet BI Sprint 1-2023,1,Sprint,Porto.pet BI Sprint 1-2023
340,Story,PPDT-1350,171942,NaN,"Inclusão do campo ""produto"" na tabela beneficiarios e custos_medicos",Jose Antonio Lopes Junior,60fff2c1eb7aef0069849e16,Bruno Pedroso Curtarelli,5c335a512020af5dbe85b958,1 - Highest,...,planejado,43200.0,43200.0,23/Jan/23 12:58 PM,NaN,Porto.pet BI Sprint 1-2023,Porto.pet BI Sprint 23,2,Sprint.1,Porto.pet BI Sprint 1-2023
351,Story,PPDT-1301,131791,NaN,Atualização de paineis legado,Jose Antonio Lopes Junior,60fff2c1eb7aef0069849e16,Bruno Pedroso Curtarelli,5c335a512020af5dbe85b958,3 - Medium,...,planejado,NaN,0.0,23/Jan/23 12:57 PM,82800.0,Porto.pet BI Sprint 1-2023,Porto.pet BI Sprint 23,2,Sprint.1,Porto.pet BI Sprint 1-2023
353,Story,PPDT-1292,105134,NaN,CUSTOS MEDICOS | Inclusão de número e data de emissão da nota fiscal,Jose Antonio Lopes Junior,60fff2c1eb7aef0069849e16,Bruno Pedroso Curtarelli,5c335a512020af5dbe85b958,1 - Highest,...,planejado,43200.0,43200.0,23/Jan/23 1:00 PM,NaN,Porto.pet BI Sprint 1-2023,Porto.pet BI Sprint 23,2,Sprint.1,Porto.pet BI Sprint 1-2023
